# Ligand ADMET and Potency (Property Prediction)

The [ADMET](https://polarishub.io/competitions/asap-discovery/antiviral-admet-2025) and [Potency](https://polarishub.io/competitions/asap-discovery/antiviral-potency-2025) Challenge of the [ASAP Discovery competition](https://polarishub.io/blog/antiviral-competition) take the shape of a property prediction task. Given the SMILES (or, to be more precise, the CXSMILES) of a molecule, you are asked to predict the numerical properties of said molecule. This is a relatively straight-forward application of ML and this notebook will quickly get you up and running!

To begin with, choose one of the two challenges! The code will look the same for both. 

In [1]:
CHALLENGE = "antiviral-admet-2025"  # or: "antiviral-potency-2025"

## Load the competition

Let's first load the competition from Polaris.

Make sure you are logged in! If not, simply run `polaris login` and follow the instructions. 

In [2]:
import polaris as po

competition = po.load_competition(f"asap-discovery/{CHALLENGE}")

2025-01-13 09:03:20.031 | INFO     | polaris._artifact:_validate_version:90 - The version of Polaris that was used to create the artifact (0.11.0) is different from the currently installed version of Polaris (0.10.1.dev5+gffca9da).
2025-01-13 09:03:20.037 | WARNING  | polaris.dataset._base:zarr_root:179 - You're loading data from a remote location. If the dataset is small enough, consider caching the dataset first using CompetitionSpecification.cache() for more performant data access.


As suggested in the logs, we'll cache the dataset. Note that this is not strictly necessary, but it does speed up later steps.

In [3]:
competition.cache()

2025-01-13 09:03:20.947 | INFO     | polaris.dataset._base:_cache_zarr:374 - Copying Zarr archive to /Users/cas.wognum/Library/Caches/polaris/datasets/dd13815d-94c3-4dfe-aee8-3fb4b3427a93/data.zarr. This may take a while.
2025-01-13 09:03:21.667 | INFO     | polaris.hub.storage:copy_to_destination:177 - Copied 16 (0.07 MiB), skipped 0, of 16 keys. 100.00% completed.


'/Users/cas.wognum/Library/Caches/polaris/datasets/dd13815d-94c3-4dfe-aee8-3fb4b3427a93'

Let's get the train and test set and take a look at the data structure.

In [4]:
train, test = competition.get_train_test_split()

In [5]:
train[0]

('COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|',
 {'MLM': nan, 'MDR1-MDCKII': 2.0, 'HLM': nan, 'KSOL': nan, 'LogD': 0.3})

In [6]:
test[0]

'CC(C)[C@H]1C2=C(CCN1C(=O)CC1=CN=CC3=CC=CC=C13)SC=C2 |o1:3|'

### Raw data dump
We've decided to sacrifice the completeness of the scientific data to improve its ease of use. For those that are interested, you can also access the raw data dump that this dataset has been created from.

In [7]:
import fsspec
import zipfile

with fsspec.open("https://fs.polarishub.io/2025-01-asap-discovery/raw_data_package.zip") as fd:
    with zipfile.ZipFile(fd, 'r') as zip_ref:
        zip_ref.extractall("./raw_data_package/")

In [8]:
import pandas as pd
from pathlib import Path

subdir = "admet" if CHALLENGE == "antiviral-admet-2025" else "potency"

path = Path("./raw_data_package")
path = path / subdir

csv_files = list(path.glob("*.csv"))
pd.read_csv(csv_files[0]).head(3)

,in-vitro_MDR1-MDCKII-Papp_bienta: mean_Papp_A_to_B (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: mean_Papp_A_to_B (Num) (10^-6 cm/s),in-vitro_MDR1-MDCKII-Papp_bienta: SD_Papp_A_to_B (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: SD_Papp_A_to_B (Num),in-vitro_MDR1-MDCKII-Papp_bienta: mean_percent_recovery_A_to_B (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: mean_percent_recovery_A_to_B (Num),in-vitro_MDR1-MDCKII-Papp_bienta: SD_percent_recovery_A_to_B (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: SD_percent_recovery_A_to_B (Num),in-vitro_MDR1-MDCKII-Papp_bienta: mean_Papp_B_to_A (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: mean_Papp_B_to_A (Num) (10^-6 cm/s),in-vitro_MDR1-MDCKII-Papp_bienta: SD_Papp_B_to_A (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: SD_Papp_B_to_A (Num),in-vitro_MDR1-MDCKII-Papp_bienta: mean_percent_recovery_B_to_A (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: mean_percent_recovery_B_to_A (Num),in-vitro_MDR1-MDCKII-Papp_bienta: SD_percent_recovery_B_to_A (Mod),in-vitro_MDR1-MDCKII-Papp_bienta: SD_percent_recovery_B_to_A (Num),Molecule Name,CXSMILES (CDD Compatible),Batch Created Date
0,=,9.46,=,0.359309,=,82.0,=,0.453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASAP-0023274,O=C(NC1=CC(Cl)=CC(C(=O)NC2=CC=CC(N3N=NNC3=O)=C...,2024-04-01
1,=,5.06,=,0.917089,=,85.5,=,2.780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASAP-0023270,O=C(NC1=CC(Cl)=CC(C(=O)NC2=CC=CC(C3=NN=NN3)=C2...,2024-04-01
2,=,2.79,=,0.024695,=,76.0,=,1.780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASAP-0023266,CC[C@H](CC1=NN=NN1)C1=CC=C(NC(=O)C2=CC(Cl)=CC(...,2024-04-01


## Build a model
Next, we'll train a simple baseline model using scikit-learn. 

You'll notice that the challenge has multiple targets.

In [9]:
train.target_cols

['MLM', 'MDR1-MDCKII', 'HLM', 'KSOL', 'LogD']

An interesting idea would be to build a multi-task model to leverage shared information across tasks.

For the sake of simplicity, however, we'll simply build a model per target here. 

In [10]:
import datamol as dm
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor

# Prepare the input data. We'll use Datamol to compute the ECFP fingerprints for both the train and test columns.
X_train = np.array([dm.to_fp(dm.to_mol(smi)) for smi in train.X])
X_test = np.array([dm.to_fp(dm.to_mol(smi)) for smi in test.X])

y_pred = {}

# For each of the targets...
for tgt in competition.target_cols:

    # We get the training targets
    # Note that we need to mask out NaNs since the multi-task matrix is sparse.
    y_true = train.y[tgt]
    mask = ~np.isnan(y_true)

    # We'll train a simple baseline model
    model = GradientBoostingRegressor()
    model.fit(X_train[mask], y_true[mask])

    # And then use that to predict the targets for the test set
    y_pred[tgt] = model.predict(X_test)

## Submit your predictions
Submitting your predictions to the competition is simple.

In [11]:
competition.submit_predictions(
    predictions=y_pred,
    prediction_name="my-first-predictions",
    prediction_owner="cwognum",
    report_url="https://www.example.com", 
    # The below metadata is optional, but recommended.
    github_url="https://github.com/polaris-hub/polaris",
    description="Just testing the Polaris API here!",
    tags=["tutorial"],
    user_attributes={"Framework": "Scikit-learn", "Method": "Gradient Boosting"}
)

✅ SUCCESS: Your competition predictions have been successfully uploaded to the Hub for evaluation.
 


/Users/cas.wognum/micromamba/envs/polaris/lib/python3.12/site-packages/yaspin/core.py:171: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


For the ASAP competition, we will only evaluate your latest submission. 

The results will only be disclosed after the competition ends.

The End.